In [2]:
# Objectives:

# FOR RELEASE 1 MVP
# - Only 2019 data
# - Rename each 2019 data column (population, rent, crime)
# - Create column with city, state (abbreviation?)
# - Column with state
# - Make table for each csv, or just join with pandas
# - Join 3 df/tables on city, state (abbreviation?) column
# - Check for duplicate cities and drop (already drop with inner join)
# - Drop any cities that lack pop, rent, crime data (already drop with inner join, but still drop NaN values and fill later)
# - Bin population, rent, and crime data
# - Categorize population, rent, and crime data

# MACHINE LEARNNG (in upcoming notebook)
# - Vectorize data
# - Train nearest neighbors model on city/state, pop, rent, and crime data
# - Make model into a function
# - Use function to make a recommendation of Location based on population, rent, crime rate
# - Check to see if recommendation matches well with data. If so:
# - Pickle the model and it is ready to be put into API and tested with Web/iOS
# - Once these steps are completed and working, we will also incorporate walkability and livability score in Release 2.
# - When walkability and livability scores are also included and working well in the model, we welcome and further additions to the model, granted the data is from 2019 (otherwise we can include a disclaimer, or we push all of the data used back to 2018, for example, as long as the data all comes from the same year)

# STRETCH GOALS
# - add more data that fit team's user stories
# - attempt forecasting using data from 2010-2020
# - try fb prophet model among other time series models and techniques
# - be in conversation with engineers

In [3]:
# Imports

import pandas as pd
import numpy as np

In [4]:
# Load an inspect population data

population = pd.read_csv("population.csv")
population

,Location,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019 Population
0,"Abbeville city, Alabama","2,688","2,705","2,699","2,694","2,643","2,628","2,608","2,600","2,584","2,575","2,571","2,560"
1,"Adamsville city, Alabama","4,522","4,506","4,500","4,493","4,471","4,449","4,420","4,390","4,356","4,327","4,308","4,281"
2,"Addison town, Alabama",758,754,751,750,743,742,739,734,731,726,723,718
3,"Akron town, Alabama",356,356,355,347,347,343,338,339,333,332,331,328
4,"Alabaster city, Alabama","30,352","31,112","31,209","31,375","31,684","31,980","32,182","32,772","33,017","33,275","33,413","33,487"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19497,"Wamsutter town, Wyoming",451,451,450,453,462,487,508,499,493,483,474,467
19498,"Wheatland town, Wyoming","3,627","3,625","3,622","3,620","3,626","3,622","3,642","3,645","3,587","3,549","3,527","3,462"
19499,"Worland city, Wyoming","5,487","5,487","5,487","5,436","5,419","5,419","5,328","5,332","5,263","5,158","5,071","5,024"
19500,"Wright town, Wyoming","1,807","1,807","1,810","1,812","1,864","1,860","1,856","1,885","1,857","1,760","1,754","1,753"


In [5]:
# Check population data types

population.dtypes

Location           object
Census             object
Estimates Base     object
2010               object
2011               object
2012               object
2013               object
2014               object
2015               object
2016               object
2017               object
2018               object
2019 Population    object
dtype: object

In [6]:
# Drop columns from population data (or can create a copy with just location and 2019 population data)

population = population.drop(['Census', 'Estimates Base', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
population

# In Location column, after the name of the town or city it contains "town" or "city". Wrangle this out.

,Location,2019 Population
0,"Abbeville city, Alabama","2,560"
1,"Adamsville city, Alabama","4,281"
2,"Addison town, Alabama",718
3,"Akron town, Alabama",328
4,"Alabaster city, Alabama","33,487"
...,...,...
19497,"Wamsutter town, Wyoming",467
19498,"Wheatland town, Wyoming","3,462"
19499,"Worland city, Wyoming","5,024"
19500,"Wright town, Wyoming","1,753"


In [7]:
# Create new column to specify if Location is city or town

population["Town or City"] = population['Location'].str.extract("(city|town)")
population

,Location,2019 Population,Town or City
0,"Abbeville city, Alabama","2,560",city
1,"Adamsville city, Alabama","4,281",city
2,"Addison town, Alabama",718,town
3,"Akron town, Alabama",328,town
4,"Alabaster city, Alabama","33,487",city
...,...,...,...
19497,"Wamsutter town, Wyoming",467,town
19498,"Wheatland town, Wyoming","3,462",town
19499,"Worland city, Wyoming","5,024",city
19500,"Wright town, Wyoming","1,753",town


In [8]:
# Remove city and town strings from Location column so that the column can be used to join with other data

population["Location"] = population['Location'].str.replace(' city', '', regex=False)
population["Location"] = population['Location'].str.replace(' town', '', regex=False)

population

# Consider adding just a State column, having City and State columns separate

,Location,2019 Population,Town or City
0,"Abbeville, Alabama","2,560",city
1,"Adamsville, Alabama","4,281",city
2,"Addison, Alabama",718,town
3,"Akron, Alabama",328,town
4,"Alabaster, Alabama","33,487",city
...,...,...,...
19497,"Wamsutter, Wyoming",467,town
19498,"Wheatland, Wyoming","3,462",town
19499,"Worland, Wyoming","5,024",city
19500,"Wright, Wyoming","1,753",town


In [9]:
# Load and inspect rental rates data

rent = pd.read_csv("rental_rates.csv")
rent

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11
0,61639,10025,1,"New York, NY",3012.0,3025.0,3037.0,3049.0,3062.0,3074.0,...,3268.0,3234.0,3199.0,3162.0,3125.0,3088.0,3048.0,3008.0,2968.0,2925.0
1,84654,60657,2,"Chicago, IL",1588.0,1594.0,1599.0,1605.0,1610.0,1615.0,...,1834.0,1829.0,1824.0,1818.0,1813.0,1807.0,1801.0,1795.0,1788.0,1781.0
2,61637,10023,3,"New York, NY",3114.0,3123.0,3131.0,3140.0,3148.0,3156.0,...,3307.0,3275.0,3244.0,3211.0,3178.0,3144.0,3108.0,3072.0,3035.0,2997.0
3,91982,77494,4,"Houston, TX",1759.0,1763.0,1766.0,1770.0,1773.0,1776.0,...,1775.0,1777.0,1780.0,1782.0,1785.0,1788.0,1791.0,1794.0,1796.0,1799.0
4,84616,60614,5,"Chicago, IL",1740.0,1745.0,1750.0,1755.0,1759.0,1764.0,...,2023.0,2017.0,2010.0,2003.0,1995.0,1988.0,1979.0,1970.0,1962.0,1952.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,62321,11976,9253,"New York, NY",63788.0,NaN,NaN,NaN,NaN,NaN,...,49083.0,48860.0,48636.0,NaN,NaN,NaN,NaN,NaN,NaN,47911.0
3186,58624,2110,9469,"Boston, MA",4113.0,4105.0,4097.0,4089.0,NaN,4077.0,...,4510.0,NaN,NaN,4350.0,4294.0,4239.0,4180.0,4121.0,4062.0,4002.0
3187,66128,20004,9592,"Washington, DC",NaN,NaN,2295.0,2304.0,NaN,2323.0,...,2496.0,2494.0,2492.0,2490.0,2487.0,2484.0,2481.0,2478.0,2475.0,2471.0
3188,399647,80951,9634,"Colorado Springs, CO",NaN,1252.0,1256.0,1260.0,1265.0,1269.0,...,NaN,1656.0,1662.0,1669.0,1676.0,1683.0,1690.0,1697.0,1704.0,1712.0


In [10]:
# Check rent data types

rent.dtypes

RegionID        int64
RegionName      int64
SizeRank        int64
MsaName        object
2014-01       float64
               ...   
2020-07       float64
2020-08       float64
2020-09       float64
2020-10       float64
2020-11       float64
Length: 87, dtype: object

In [11]:
# Drop all columns you dont want, or just use code below to make new df

#rent = rent.drop(['RegionID', 'RegionName', 'SizeRank', '2014-01', '2014-02', '2014-03', '2014-04', ...], axis=1)

In [12]:
# Create copy of rental rates dataframe and change column names

rent = rent[['MsaName','2019-12']].copy()
rent = rent.rename(columns = {"MsaName":"Location"})
rent = rent.rename(columns = {"2019-12":"2019 Rental Rates"})
rent

,Location,2019 Rental Rates
0,"New York, NY",3311.0
1,"Chicago, IL",1838.0
2,"New York, NY",3344.0
3,"Houston, TX",1772.0
4,"Chicago, IL",2028.0
...,...,...
3185,"New York, NY",50175.0
3186,"Boston, MA",4585.0
3187,"Washington, DC",2495.0
3188,"Colorado Springs, CO",1640.0


In [13]:
# Replace state abbreviations with full state names

rent["Location"] = rent['Location'].str.replace('AK', 'Alaska', regex=False)
rent["Location"] = rent['Location'].str.replace('AL', 'Alabama', regex=False)
rent["Location"] = rent['Location'].str.replace('AR', 'Arkansas', regex=False)
rent["Location"] = rent['Location'].str.replace('AS', 'American Samoa', regex=False)
rent["Location"] = rent['Location'].str.replace('AZ', 'Arizona', regex=False)
rent["Location"] = rent['Location'].str.replace('CA', 'California', regex=False)
rent["Location"] = rent['Location'].str.replace('CO', 'Colorado', regex=False)
rent["Location"] = rent['Location'].str.replace('CT', 'Connecticut', regex=False)
rent["Location"] = rent['Location'].str.replace('DC', 'District of Columbia', regex=False)
rent["Location"] = rent['Location'].str.replace('DE', 'Delaware', regex=False)
rent["Location"] = rent['Location'].str.replace('FL', 'Florida', regex=False)
rent["Location"] = rent['Location'].str.replace('GA', 'Georgia', regex=False)
rent["Location"] = rent['Location'].str.replace('GU', 'Guam', regex=False)
rent["Location"] = rent['Location'].str.replace('HI', 'Hawaii', regex=False)
rent["Location"] = rent['Location'].str.replace('IA', 'Iowa', regex=False)
rent["Location"] = rent['Location'].str.replace('ID', 'Idaho', regex=False)
rent["Location"] = rent['Location'].str.replace('IL', 'Illinois', regex=False)
rent["Location"] = rent['Location'].str.replace('IN', 'Indiana', regex=False)
rent["Location"] = rent['Location'].str.replace('KS', 'Kansas', regex=False)
rent["Location"] = rent['Location'].str.replace('KY', 'Kentucky', regex=False)
rent["Location"] = rent['Location'].str.replace('LA', 'Louisiana', regex=False)
rent["Location"] = rent['Location'].str.replace('MA', 'Massachusetts', regex=False)
rent["Location"] = rent['Location'].str.replace('MD', 'Maryland', regex=False)
rent["Location"] = rent['Location'].str.replace('ME', 'Maine', regex=False)
rent["Location"] = rent['Location'].str.replace('MI', 'Michigan', regex=False)
rent["Location"] = rent['Location'].str.replace('MN', 'Minnesota', regex=False)
rent["Location"] = rent['Location'].str.replace('MO', 'Missouri', regex=False)
rent["Location"] = rent['Location'].str.replace('MP', 'Northern Mariana Islands', regex=False)
rent["Location"] = rent['Location'].str.replace('MS', 'Mississippi', regex=False)
rent["Location"] = rent['Location'].str.replace('MT', 'Montana', regex=False)
rent["Location"] = rent['Location'].str.replace('NA', 'National', regex=False)
rent["Location"] = rent['Location'].str.replace('NC', 'North Carolina', regex=False)
rent["Location"] = rent['Location'].str.replace('ND', 'North Dakota', regex=False)
rent["Location"] = rent['Location'].str.replace('NE', 'Nebraska', regex=False)
rent["Location"] = rent['Location'].str.replace('NH', 'New Hampshire', regex=False)
rent["Location"] = rent['Location'].str.replace('NJ', 'New Jersey', regex=False)
rent["Location"] = rent['Location'].str.replace('NM', 'New Mexico', regex=False)
rent["Location"] = rent['Location'].str.replace('NV', 'Nevada', regex=False)
rent["Location"] = rent['Location'].str.replace('NY', 'New York', regex=False)
rent["Location"] = rent['Location'].str.replace('OH', 'Ohio', regex=False)
rent["Location"] = rent['Location'].str.replace('OK', 'Oklahoma', regex=False)
rent["Location"] = rent['Location'].str.replace('OR', 'Oregon', regex=False)
rent["Location"] = rent['Location'].str.replace('PA', 'Pennsylvania', regex=False)
rent["Location"] = rent['Location'].str.replace('PR', 'Puerto Rico', regex=False)
rent["Location"] = rent['Location'].str.replace('RI', 'Rhode Island', regex=False)
rent["Location"] = rent['Location'].str.replace('SC', 'South Carolina', regex=False)
rent["Location"] = rent['Location'].str.replace('SD', 'South Dakota', regex=False)
rent["Location"] = rent['Location'].str.replace('TN', 'Tennessee', regex=False)
rent["Location"] = rent['Location'].str.replace('TX', 'Texas', regex=False)
rent["Location"] = rent['Location'].str.replace('UT', 'Utah', regex=False)
rent["Location"] = rent['Location'].str.replace('VA', 'Virginia', regex=False)
rent["Location"] = rent['Location'].str.replace('VI', 'Virgin Islands', regex=False)
rent["Location"] = rent['Location'].str.replace('VT', 'Vermont', regex=False)
rent["Location"] = rent['Location'].str.replace('WA', 'Washington', regex=False)
rent["Location"] = rent['Location'].str.replace('WI', 'Wisconsin', regex=False)
rent["Location"] = rent['Location'].str.replace('WV', 'West Virginia', regex=False)
rent["Location"] = rent['Location'].str.replace('WY', 'Wyoming', regex=False)
rent

,Location,2019 Rental Rates
0,"New York, New York",3311.0
1,"Chicago, Illinois",1838.0
2,"New York, New York",3344.0
3,"Houston, Texas",1772.0
4,"Chicago, Illinois",2028.0
...,...,...
3185,"New York, New York",50175.0
3186,"Boston, Massachusetts",4585.0
3187,"Washington, District of Columbia",2495.0
3188,"Colorado Springs, Colorado",1640.0


In [14]:
# Load and inspect crime rates data (need to replicate state for every city)

crime = pd.read_csv("crime_rates.csv")
crime

,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,Alabama,Hoover,"85,670",114,4.0,15,27,68,"1,922",128,"1,694",100,2
1,Alaska,Anchorage,"287,731","3,581",32.0,540,621,"2,388","12,261","1,692","9,038","1,531",93
2,Alaska,Bethel,"6,544",130,1.0,47,3,79,132,20,84,28,12
3,Alaska,Bristol Bay Borough,852,2,0.0,0,0,2,20,5,8,7,0
4,Alaska,Cordova,"2,150",0,0.0,0,0,0,7,1,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,Wyoming,Sheridan,"17,895",9,0.0,4,0,5,369,75,278,16,3
8101,Wyoming,Thermopolis,"2,830",13,0.0,0,0,13,34,7,22,5,0
8102,Wyoming,Torrington,"6,709",13,0.0,4,1,8,48,8,40,0,0
8103,Wyoming,Wheatland,"3,544",7,0.0,1,0,6,72,24,45,3,0


In [15]:
# Check crime data types

crime.dtypes

State                                    object
City                                     object
Population                               object
Violent crime                            object
Murder and nonnegligent manslaughter    float64
Rape                                     object
Robbery                                  object
Aggravated assault                       object
Property crime                           object
Burglary                                 object
Larceny-theft                            object
Motor vehicle theft                      object
Arson                                    object
dtype: object

In [16]:
# Make new Location column with City and State data, so the data can be joineed with the other data on this column

crime['Location'] = crime['City'] + ',' + ' ' + crime['State']
crime

# Consider adding Alabama crime data from 2018

,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Location
0,Alabama,Hoover,"85,670",114,4.0,15,27,68,"1,922",128,"1,694",100,2,"Hoover, Alabama"
1,Alaska,Anchorage,"287,731","3,581",32.0,540,621,"2,388","12,261","1,692","9,038","1,531",93,"Anchorage, Alaska"
2,Alaska,Bethel,"6,544",130,1.0,47,3,79,132,20,84,28,12,"Bethel, Alaska"
3,Alaska,Bristol Bay Borough,852,2,0.0,0,0,2,20,5,8,7,0,"Bristol Bay Borough, Alaska"
4,Alaska,Cordova,"2,150",0,0.0,0,0,0,7,1,6,0,0,"Cordova, Alaska"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,Wyoming,Sheridan,"17,895",9,0.0,4,0,5,369,75,278,16,3,"Sheridan, Wyoming"
8101,Wyoming,Thermopolis,"2,830",13,0.0,0,0,13,34,7,22,5,0,"Thermopolis, Wyoming"
8102,Wyoming,Torrington,"6,709",13,0.0,4,1,8,48,8,40,0,0,"Torrington, Wyoming"
8103,Wyoming,Wheatland,"3,544",7,0.0,1,0,6,72,24,45,3,0,"Wheatland, Wyoming"


In [17]:
# When wrangling is done here, combine tables, bin data in new columns, and push data to PG DB. 
 
# Example of Joins

merged_population_rent = pd.merge(left=population, right=rent, left_on='Location', right_on='Location')
merged_population_rent

,Location,2019 Population,Town or City,2019 Rental Rates
0,"Birmingham, Alabama","209,403",city,935.0
1,"Birmingham, Alabama","209,403",city,1328.0
2,"Birmingham, Alabama","209,403",city,1675.0
3,"Birmingham, Alabama","209,403",city,966.0
4,"Birmingham, Alabama","209,403",city,1426.0
...,...,...,...,...
2502,"Milwaukee, Wisconsin","590,157",city,839.0
2503,"Milwaukee, Wisconsin","590,157",city,948.0
2504,"Milwaukee, Wisconsin","590,157",city,973.0
2505,"Milwaukee, Wisconsin","590,157",city,792.0


In [18]:
# Merge population, rent, and crime data which can be pushed to DB. Data can also be added to this, more data preserved, and binned.

merged_population_rent_crime = pd.merge(left=merged_population_rent, right=crime, left_on='Location', right_on='Location')
merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,"Phoenix, Arizona","1,680,992",city,1424.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
1,"Phoenix, Arizona","1,680,992",city,1522.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
2,"Phoenix, Arizona","1,680,992",city,1461.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
3,"Phoenix, Arizona","1,680,992",city,1549.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
4,"Phoenix, Arizona","1,680,992",city,1611.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin","590,157",city,839.0,Wisconsin,Milwaukee,"590,923","7,874",97.0,427,"1,911","5,439","15,097","3,594","8,053","3,450",198
2046,"Milwaukee, Wisconsin","590,157",city,948.0,Wisconsin,Milwaukee,"590,923","7,874",97.0,427,"1,911","5,439","15,097","3,594","8,053","3,450",198
2047,"Milwaukee, Wisconsin","590,157",city,973.0,Wisconsin,Milwaukee,"590,923","7,874",97.0,427,"1,911","5,439","15,097","3,594","8,053","3,450",198
2048,"Milwaukee, Wisconsin","590,157",city,792.0,Wisconsin,Milwaukee,"590,923","7,874",97.0,427,"1,911","5,439","15,097","3,594","8,053","3,450",198


In [19]:
# Check data types before binning

merged_population_rent_crime.dtypes

Location                                 object
2019 Population                          object
Town or City                             object
2019 Rental Rates                       float64
State                                    object
City                                     object
Population                               object
Violent crime                            object
Murder and nonnegligent manslaughter    float64
Rape                                     object
Robbery                                  object
Aggravated assault                       object
Property crime                           object
Burglary                                 object
Larceny-theft                            object
Motor vehicle theft                      object
Arson                                    object
dtype: object

In [20]:
# Remove commas from columns and change data type to floats

merged_population_rent_crime['2019 Population'] = merged_population_rent_crime['2019 Population'].replace(',','', regex=True)
merged_population_rent_crime['Population'] = merged_population_rent_crime['Population'].replace(',','', regex=True)
merged_population_rent_crime['Violent crime'] = merged_population_rent_crime['Violent crime'].replace(',','', regex=True)
merged_population_rent_crime['Murder and nonnegligent manslaughter'] = merged_population_rent_crime['Murder and nonnegligent manslaughter'].replace(',','', regex=True)
merged_population_rent_crime['Rape'] = merged_population_rent_crime['Rape'].replace(',','', regex=True)
merged_population_rent_crime['Robbery'] = merged_population_rent_crime['Robbery'].replace(',','', regex=True)
merged_population_rent_crime['Aggravated assault'] = merged_population_rent_crime['Aggravated assault'].replace(',','', regex=True)
merged_population_rent_crime['Property crime'] = merged_population_rent_crime['Property crime'].replace(',','', regex=True)
merged_population_rent_crime['Burglary'] = merged_population_rent_crime['Burglary'].replace(',','', regex=True)
merged_population_rent_crime['Larceny-theft'] = merged_population_rent_crime['Larceny-theft'].replace(',','', regex=True)
merged_population_rent_crime['Motor vehicle theft'] = merged_population_rent_crime['Motor vehicle theft'].replace(',','', regex=True)
merged_population_rent_crime['Arson'] = merged_population_rent_crime['Arson'].replace(',','', regex=True)

merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,"Phoenix, Arizona",1680992,city,1424.0,Arizona,Phoenix,1688722,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201
1,"Phoenix, Arizona",1680992,city,1522.0,Arizona,Phoenix,1688722,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201
2,"Phoenix, Arizona",1680992,city,1461.0,Arizona,Phoenix,1688722,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201
3,"Phoenix, Arizona",1680992,city,1549.0,Arizona,Phoenix,1688722,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201
4,"Phoenix, Arizona",1680992,city,1611.0,Arizona,Phoenix,1688722,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839.0,Wisconsin,Milwaukee,590923,7874,97.0,427,1911,5439,15097,3594,8053,3450,198
2046,"Milwaukee, Wisconsin",590157,city,948.0,Wisconsin,Milwaukee,590923,7874,97.0,427,1911,5439,15097,3594,8053,3450,198
2047,"Milwaukee, Wisconsin",590157,city,973.0,Wisconsin,Milwaukee,590923,7874,97.0,427,1911,5439,15097,3594,8053,3450,198
2048,"Milwaukee, Wisconsin",590157,city,792.0,Wisconsin,Milwaukee,590923,7874,97.0,427,1911,5439,15097,3594,8053,3450,198


In [21]:
# Check for null values

merged_population_rent_crime.isnull().sum().sum()

440

In [22]:
# Drop all null values for now, can return to preserve rows by filling (fillna(), replace() and interpolate())

merged_population_rent_crime = merged_population_rent_crime.dropna()
merged_population_rent_crime.shape

(1626, 17)

In [23]:
# Change datatype to float or int, int looks better with no decimal and trailing 0

#merged_population_rent_crime['2019 Population'].astype('float64')
#merged_population_rent_crime['2019 Rental Rates'].astype('float64')
#merged_population_rent_crime['Population'].astype('float64')
#merged_population_rent_crime['Violent crime'].astype('float64')
#merged_population_rent_crime['Murder and nonnegligent manslaughter'].astype('float64')
#merged_population_rent_crime['Rape'].astype('float64')
#merged_population_rent_crime['Robbery'].astype('float64')
#merged_population_rent_crime['Aggravated assault'].astype('float64')
#merged_population_rent_crime['Property crime'].astype('float64')
#merged_population_rent_crime['Burglary'].astype('float64')
#merged_population_rent_crime['Larceny-theft'].astype('float64')
#merged_population_rent_crime['Motor vehicle theft'].astype('float64')
#merged_population_rent_crime['Arson'].astype('float64')

merged_population_rent_crime['2019 Population'] = merged_population_rent_crime['2019 Population'].astype('int')
merged_population_rent_crime['2019 Rental Rates'] = merged_population_rent_crime['2019 Rental Rates'].astype('int') # ValueError: Cannot convert non-finite values (NA or inf) to integer
merged_population_rent_crime['Population'] = merged_population_rent_crime['Population'].astype('int')
merged_population_rent_crime['Violent crime'] = merged_population_rent_crime['Violent crime'].astype('int')
merged_population_rent_crime['Murder and nonnegligent manslaughter'] = merged_population_rent_crime['Murder and nonnegligent manslaughter'].astype('int')
merged_population_rent_crime['Rape'] = merged_population_rent_crime['Rape'].astype('int')
merged_population_rent_crime['Robbery'] = merged_population_rent_crime['Robbery'].astype('int')
merged_population_rent_crime['Aggravated assault'] = merged_population_rent_crime['Aggravated assault'].astype('int')
merged_population_rent_crime['Property crime'] = merged_population_rent_crime['Property crime'].astype('int') # ValueError: cannot convert float NaN to integer
merged_population_rent_crime['Burglary'] = merged_population_rent_crime['Burglary'].astype('int') # ValueError: cannot convert float NaN to integer
merged_population_rent_crime['Larceny-theft'] = merged_population_rent_crime['Larceny-theft'].astype('int')
merged_population_rent_crime['Motor vehicle theft'] = merged_population_rent_crime['Motor vehicle theft'].astype('int')
merged_population_rent_crime['Arson'] = merged_population_rent_crime['Arson'].astype('int') # ValueError: cannot convert float NaN to integer

# These columns previously had NA or NaN values which had to be dropped. These columns can be revisited to preserve more data.

#merged_population_rent_crime['2019 Rental Rates']
#merged_population_rent_crime['Property crime']
#merged_population_rent_crime['Burglary']
#merged_population_rent_crime['Arson']

In [24]:
# Check data types again before binning, since changing

merged_population_rent_crime.dtypes

Location                                object
2019 Population                          int32
Town or City                            object
2019 Rental Rates                        int32
State                                   object
City                                    object
Population                               int32
Violent crime                            int32
Murder and nonnegligent manslaughter     int32
Rape                                     int32
Robbery                                  int32
Aggravated assault                       int32
Property crime                           int32
Burglary                                 int32
Larceny-theft                            int32
Motor vehicle theft                      int32
Arson                                    int32
dtype: object

In [43]:
# Combine crime data into new column and divide by population * 1000 (This is how crime rate is calculated)

merged_population_rent_crime['Crime Rate']=merged_population_rent_crime.iloc[:,-11:].sum(axis=1)/merged_population_rent_crime['Population']*100000

#merged_population_rent_crime['Crime Rate'] = merged_population_rent_crime['Violent crime'] + merged_population_rent_crime['Murder and nonnegligent manslaughter'] + #merged_population_rent_crime['Rape'] + merged_population_rent_crime + merged_population_rent_crime['Robbery'] + ['Aggravated assault'] + merged_population_rent_crime['Property #crime'] + merged_population_rent_crime['Burglary'] + merged_population_rent_crime['Larceny-theft'] + merged_population_rent_crime['Motor vehicle theft'] + #merged_population_rent_crime['Arson']/merged_population_rent_crime['Population']*100000

#merged_population_rent_crime['Crime Rate'] = total_crimes/merged_population_rent_crime['Population']*100000

merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,...,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Rate,Urban Population by City Size,Rental Rate Categories,Urban Population by City Size Categories,Urban Population by City Size Ranges,Rental Rate Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,6641.065278,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549"
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,6641.065278,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549"
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,6641.065278,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549"
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,6641.065278,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549"
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,6641.065278,Large Metropolitan Area,Higher Rent,Large Metropolitan Area,"1,500,000 <","$1,549-1,891.50"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,590923,7874,97,427,...,3594,8053,3450,198,5143.180054,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50"
2046,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,590923,7874,97,427,...,3594,8053,3450,198,5143.180054,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50"
2047,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,590923,7874,97,427,...,3594,8053,3450,198,5143.180054,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50"
2048,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,590923,7874,97,427,...,3594,8053,3450,198,5143.180054,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50"


In [26]:
# Describe the data to begin binning data with pandas cut or qcut

merged_population_rent_crime['2019 Population'].describe()

count    1.626000e+03
mean     8.890544e+05
std      7.568980e+05
min      4.927100e+04
25%      3.313600e+05
50%      6.547410e+05
75%      1.021795e+06
max      2.693976e+06
Name: 2019 Population, dtype: float64

In [27]:
# Quartile cut the data, to see how it is evenly distributed

pd.qcut(merged_population_rent_crime['2019 Population'], q=6)

0       (881549.0, 1680992.0]
1       (881549.0, 1680992.0]
2       (881549.0, 1680992.0]
3       (881549.0, 1680992.0]
4       (881549.0, 1680992.0]
                ...          
2045     (399700.0, 654741.0]
2046     (399700.0, 654741.0]
2047     (399700.0, 654741.0]
2048     (399700.0, 654741.0]
2049     (399700.0, 654741.0]
Name: 2019 Population, Length: 1626, dtype: category
Categories (6, interval[float64]): [(49270.999, 259680.0] < (259680.0, 399700.0] < (399700.0, 654741.0] < (654741.0, 881549.0] < (881549.0, 1680992.0] < (1680992.0, 2693976.0]]

In [40]:
# Bin 2019 population into Urban population by City Size Categories and ranges

# (https://data.oecd.org/popregion/urban-population-by-city-size.htm#:~:text=their%20administrative%20boundaries.-,Urban%20areas%20in%20OECD%20countries%20are%20classified%20as%3A%20large%20metropolitan,areas%20if%20their%20population%20is)

# Urban areas in OECD countries are classified as: large metropolitan areas if they have a population of 1.5 million or more; metropolitan areas if their population is between 500 000 and 1.5 million; medium-size urban areas if their population is between 200 000 and 500 000; and, small urban areas if their population is between 50 000 and 200 000. This indicator is measured as a percentage of the national population.

# Qcut

#merged_population_rent_crime['Urban Population by City Size'] = pd.qcut(merged_population_rent_crime['2019 Population'], q=4, labels = ["Small Urban Area", "Medium-size Urban Area", "Metropolitan Area", "Large Metropolitan Area"])

#merged_population_rent_crime['Population by City Size'] = pd.qcut(merged_population_rent_crime['2019 Population'], q=4)

#merged_population_rent_crime

# Consider using just cut to fit the labels above. qcut will make the bins equal, but it is difficult to categorize cities based on those numbers.

bins = [0, 50000, 200000, 500000, 1500000, 100000000]
labels = ["Town","Small Urban Area", "Medium-size Urban Area", "Metropolitan Area", "Large Metropolitan Area"]
merged_population_rent_crime['Urban Population by City Size Categories'] = pd.cut(merged_population_rent_crime['2019 Population'], bins=bins, labels=labels)

bins = [0, 50000, 200000, 500000, 1500000, 100000000]
labels = ["0-50,000", "50,000-200,000", "200,000-500,000", "500,000-1,500,000", "1,500,000 <"]
merged_population_rent_crime['Urban Population by City Size Ranges'] = pd.cut(merged_population_rent_crime['2019 Population'], bins=bins, labels=labels)


merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,...,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Rate,Urban Population by City Size,Rental Rate Categories,Urban Population by City Size Categories,Urban Population by City Size Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,1688722,11803,131,1139,...,55974,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <"
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,1688722,11803,131,1139,...,55974,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <"
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,1688722,11803,131,1139,...,55974,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <"
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,1688722,11803,131,1139,...,55974,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <"
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,1688722,11803,131,1139,...,55974,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Higher Rent,Large Metropolitan Area,"1,500,000 <"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,590923,7874,97,427,...,15097,3594,8053,3450,198,107808.123901,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000"
2046,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,590923,7874,97,427,...,15097,3594,8053,3450,198,107808.123901,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000"
2047,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,590923,7874,97,427,...,15097,3594,8053,3450,198,107808.123901,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000"
2048,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,590923,7874,97,427,...,15097,3594,8053,3450,198,107808.123901,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000"


In [29]:
# Describe the data to begin binning data with pandas cut or qcut

merged_population_rent_crime['2019 Rental Rates'].describe()

count    1626.000000
mean     1674.565806
std       607.902951
min       582.000000
25%      1294.500000
50%      1549.000000
75%      1891.500000
max      6373.000000
Name: 2019 Rental Rates, dtype: float64

In [30]:
# Quartile cut the 2019 Rental Rates data, to see how it is evenly distributed

pd.qcut(merged_population_rent_crime['2019 Rental Rates'], q=4)

0        (1294.5, 1549.0]
1        (1294.5, 1549.0]
2        (1294.5, 1549.0]
3        (1294.5, 1549.0]
4        (1549.0, 1891.5]
              ...        
2045    (581.999, 1294.5]
2046    (581.999, 1294.5]
2047    (581.999, 1294.5]
2048    (581.999, 1294.5]
2049    (581.999, 1294.5]
Name: 2019 Rental Rates, Length: 1626, dtype: category
Categories (4, interval[float64]): [(581.999, 1294.5] < (1294.5, 1549.0] < (1549.0, 1891.5] < (1891.5, 6373.0]]

In [41]:
# Bin the 2019 Rental Rates data into categories and ranges

bins = [0, 581.99, 1294.5, 1549, 1891.5, 6373]
labels = ["Lowest Rent","Low Rent", "Average Rent", "Higher Rent", "Highest Rent"]
merged_population_rent_crime['Rental Rate Categories'] = pd.cut(merged_population_rent_crime['2019 Rental Rates'], bins=bins, labels=labels)

bins = [0, 581.99, 1294.5, 1549, 1891.5, 6373]
labels = ["$0-581.99","$581.99-1,294.50", "$1,294.50-1,549", "$1,549-1,891.50", "$1891.50-6,373"]
merged_population_rent_crime['Rental Rate Ranges'] = pd.cut(merged_population_rent_crime['2019 Rental Rates'], bins=bins, labels=labels)

merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,...,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Rate,Urban Population by City Size,Rental Rate Categories,Urban Population by City Size Categories,Urban Population by City Size Ranges,Rental Rate Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549"
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549"
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549"
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549"
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,1688722,11803,131,1139,...,9471,39427,7076,201,108038.919372,Large Metropolitan Area,Higher Rent,Large Metropolitan Area,"1,500,000 <","$1,549-1,891.50"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,590923,7874,97,427,...,3594,8053,3450,198,107808.123901,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50"
2046,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,590923,7874,97,427,...,3594,8053,3450,198,107808.123901,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50"
2047,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,590923,7874,97,427,...,3594,8053,3450,198,107808.123901,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50"
2048,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,590923,7874,97,427,...,3594,8053,3450,198,107808.123901,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50"


In [44]:
# Describe the data to begin binning data with pandas cut or qcut

merged_population_rent_crime['Crime Rate'].describe()

count     1626.000000
mean      7193.825490
std       2428.269709
min       1703.092498
25%       5897.911474
50%       7018.233292
75%       8724.805206
max      12422.929229
Name: Crime Rate, dtype: float64

In [48]:
# Quartile cut the data, to see how it is evenly distributed

pd.qcut(merged_population_rent_crime['Crime Rate'], q=4)

0                 (5897.911, 7018.233]
1                 (5897.911, 7018.233]
2                 (5897.911, 7018.233]
3                 (5897.911, 7018.233]
4                 (5897.911, 7018.233]
                     ...              
2045    (1703.0910000000001, 5897.911]
2046    (1703.0910000000001, 5897.911]
2047    (1703.0910000000001, 5897.911]
2048    (1703.0910000000001, 5897.911]
2049    (1703.0910000000001, 5897.911]
Name: Crime Rate, Length: 1626, dtype: category
Categories (4, interval[float64]): [(1703.0910000000001, 5897.911] < (5897.911, 7018.233] < (7018.233, 8724.805] < (8724.805, 12422.929]]

In [49]:
# Bin the 2019 Crimes Rate data into categories and ranges

bins = [0, 1703.0910000000001, 5897.911, 7018.233, 8724.805, 12422.929]
labels = ["Lowest Crime","Lower Crime", "Average Crime", "Higher Crime", "Highest Crime"]
merged_population_rent_crime['Crime Rate Categories'] = pd.cut(merged_population_rent_crime['Crime Rate'], bins=bins, labels=labels)

bins = [0, 1703.0910000000001, 5897.911, 7018.233, 8724.805, 12422.929]
labels = ["0-1,703.09","1,703.09-5,897.91", "5,897.91-7,018.23", "7018.23-8,724.80", "8,724.80-12,422.92"]
merged_population_rent_crime['Crime Rate Ranges'] = pd.cut(merged_population_rent_crime['Crime Rate'], bins=bins, labels=labels)

merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,...,Motor vehicle theft,Arson,Crime Rate,Urban Population by City Size,Rental Rate Categories,Urban Population by City Size Categories,Urban Population by City Size Ranges,Rental Rate Ranges,Crime Rate Categories,Crime Rate Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,1688722,11803,131,1139,...,7076,201,6641.065278,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549",Average Crime,"5,897.91-7,018.23"
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,1688722,11803,131,1139,...,7076,201,6641.065278,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549",Average Crime,"5,897.91-7,018.23"
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,1688722,11803,131,1139,...,7076,201,6641.065278,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549",Average Crime,"5,897.91-7,018.23"
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,1688722,11803,131,1139,...,7076,201,6641.065278,Large Metropolitan Area,Average Rent,Large Metropolitan Area,"1,500,000 <","$1,294.50-1,549",Average Crime,"5,897.91-7,018.23"
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,1688722,11803,131,1139,...,7076,201,6641.065278,Large Metropolitan Area,Higher Rent,Large Metropolitan Area,"1,500,000 <","$1,549-1,891.50",Average Crime,"5,897.91-7,018.23"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,590923,7874,97,427,...,3450,198,5143.180054,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50",Lower Crime,"1,703.09-5,897.91"
2046,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,590923,7874,97,427,...,3450,198,5143.180054,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50",Lower Crime,"1,703.09-5,897.91"
2047,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,590923,7874,97,427,...,3450,198,5143.180054,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50",Lower Crime,"1,703.09-5,897.91"
2048,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,590923,7874,97,427,...,3450,198,5143.180054,Metropolitan Area,Low Rent,Metropolitan Area,"500,000-1,500,000","$581.99-1,294.50",Lower Crime,"1,703.09-5,897.91"


In [50]:
# Export merged_population_rent_crime with binned categories for csv and beginning modeling in another notebook

merged_population_rent_crime.to_csv('pop_rent_crime_bins.csv')